In [91]:
# importing pandas and numpy and regex

import pandas as pd

import numpy as np

import re

In [106]:
# importing mock traffic data

df_traffic = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 203_Everyone Loves a Toll Booth\mocktraffic.csv')

df_traffic.head()

,id,plate_id,time_node_1,time_node_2,time_node_3,time_node_4,time_node_5,time_node_6
0,0,1N6AD0C,7:00:03,7:11:02,NaN,7:42:11,NaN,8:44:01
1,1,KNDMG4C,7:00:05,NaN,7:47:46,NaN,8:43:54,9:12:38
2,2,5LMJJ3J,7:00:08,7:08:12,NaN,NaN,8:15:32,8:42:42
3,3,WAUDF78,7:00:12,NaN,7:47:53,NaN,8:44:54,9:13:59
4,4,1FAHP3E,7:00:56,NaN,7:52:36,NaN,8:54:49,9:25:31


In [107]:
# importing toll data

df_toll = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 203_Everyone Loves a Toll Booth\tolls.csv')

df_toll.head()

,Segment,Toll
0,"1,2",2.25
1,"1,3",2.25
2,"2,5",3.00
3,"2,4",1.75
4,"5,6",2.25


In [108]:
# transposing traffic data

traffic_transposed = pd.melt(df_traffic, id_vars = ['id', 'plate_id'])

traffic_transposed.head()

,id,plate_id,variable,value
0,0,1N6AD0C,time_node_1,7:00:03
1,1,KNDMG4C,time_node_1,7:00:05
2,2,5LMJJ3J,time_node_1,7:00:08
3,3,WAUDF78,time_node_1,7:00:12
4,4,1FAHP3E,time_node_1,7:00:56


In [109]:
# parsing time node number

traffic_transposed['variable'] = traffic_transposed['variable'].apply(lambda x: pd.Series(re.split(r'_', x)))[[2]]

traffic_transposed.head()

,id,plate_id,variable,value
0,0,1N6AD0C,1,7:00:03
1,1,KNDMG4C,1,7:00:05
2,2,5LMJJ3J,1,7:00:08
3,3,WAUDF78,1,7:00:12
4,4,1FAHP3E,1,7:00:56


In [110]:
# 2700 nulls in value

traffic_transposed.isnull().sum()

id             0
plate_id       0
variable       0
value       2700
dtype: int64

In [111]:
# filtering out null values 

traffic_filtered = traffic_transposed[~traffic_transposed['value'].isna()]

traffic_filtered.head()

,id,plate_id,variable,value
0,0,1N6AD0C,1,7:00:03
1,1,KNDMG4C,1,7:00:05
2,2,5LMJJ3J,1,7:00:08
3,3,WAUDF78,1,7:00:12
4,4,1FAHP3E,1,7:00:56


In [112]:
# no more nulls in value

traffic_filtered.isnull().sum()

id          0
plate_id    0
variable    0
value       0
dtype: int64

In [113]:
# formatting value to datetime

traffic_filtered['value'] = pd.to_datetime(traffic_filtered['value'], format = '%H:%M:%S')

traffic_filtered.head()

C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_22624\1632098133.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_filtered['value'] = pd.to_datetime(traffic_filtered['value'], format = '%H:%M:%S')


,id,plate_id,variable,value
0,0,1N6AD0C,1,1900-01-01 07:00:03
1,1,KNDMG4C,1,1900-01-01 07:00:05
2,2,5LMJJ3J,1,1900-01-01 07:00:08
3,3,WAUDF78,1,1900-01-01 07:00:12
4,4,1FAHP3E,1,1900-01-01 07:00:56


In [124]:
# creating function to difference in minutes 

def time_diff (group):
    
    group['previous_time'] = group['value'].shift(1)
    
    group['time'] = round((group['value'] - group['previous_time']).dt.total_seconds() / 60, 0)
    
    return group

In [125]:
# sorting df by id, variable, and time and resetting index

traffic_filtered.sort_values(by = ['id', 'variable', 'value'], inplace = True)

traffic_filtered.reset_index(drop = True, inplace = True)

traffic_filtered = traffic_filtered[['id', 'plate_id', 'variable', 'value']]

traffic_filtered.head()

,id,plate_id,variable,value
0,0,1N6AD0C,1,1900-01-01 07:00:03
1,0,1N6AD0C,2,1900-01-01 07:11:02
2,0,1N6AD0C,4,1900-01-01 07:42:11
3,0,1N6AD0C,6,1900-01-01 08:44:01
4,1,KNDMG4C,1,1900-01-01 07:00:05


In [130]:
# applying function

traffic_time = traffic_filtered.groupby('id').apply(time_diff).reset_index(drop = True)


traffic_time.head(10)

,id,plate_id,variable,value,time,previous_time
0,0,1N6AD0C,1,1900-01-01 07:00:03,NaN,NaT
1,0,1N6AD0C,2,1900-01-01 07:11:02,11.0,1900-01-01 07:00:03
2,0,1N6AD0C,4,1900-01-01 07:42:11,31.0,1900-01-01 07:11:02
3,0,1N6AD0C,6,1900-01-01 08:44:01,62.0,1900-01-01 07:42:11
4,1,KNDMG4C,1,1900-01-01 07:00:05,NaN,NaT
5,1,KNDMG4C,3,1900-01-01 07:47:46,48.0,1900-01-01 07:00:05
6,1,KNDMG4C,5,1900-01-01 08:43:54,56.0,1900-01-01 07:47:46
7,1,KNDMG4C,6,1900-01-01 09:12:38,29.0,1900-01-01 08:43:54
8,2,5LMJJ3J,1,1900-01-01 07:00:08,NaN,NaT
9,2,5LMJJ3J,2,1900-01-01 07:08:12,8.0,1900-01-01 07:00:08


In [196]:
# creating function to concat current and succeeding variables

def concat_var (group):
    
    toll_paths = []
    
    previous_var = group['variable'].shift(1)
    
    toll_paths = [(str(previous if pd.notna(previous) else ' ')) + ',' + str(current)
                 for previous, current in zip(previous_var, group['variable'])]
    
    group['toll_path'] = [None] + toll_paths[1:]
    
    return group

In [197]:
# applying function

traffic_path = traffic_time.groupby('id').apply(concat_var).reset_index(drop = True)

traffic_path.head()

,id,plate_id,variable,value,time,previous_time,toll_path
0,0,1N6AD0C,1,1900-01-01 07:00:03,NaN,NaT,None
1,0,1N6AD0C,2,1900-01-01 07:11:02,11.0,1900-01-01 07:00:03,"1,2"
2,0,1N6AD0C,4,1900-01-01 07:42:11,31.0,1900-01-01 07:11:02,"2,4"
3,0,1N6AD0C,6,1900-01-01 08:44:01,62.0,1900-01-01 07:42:11,"4,6"
4,1,KNDMG4C,1,1900-01-01 07:00:05,NaN,NaT,None


In [199]:
# joining to toll df

df_merged = pd.merge(traffic_path, df_toll, right_on = 'Segment', left_on = 'toll_path')

df_merged.head()

,id,plate_id,variable,value,time,previous_time,toll_path,Segment,Toll
0,0,1N6AD0C,2,1900-01-01 07:11:02,11.0,1900-01-01 07:00:03,"1,2","1,2",2.25
1,2,5LMJJ3J,2,1900-01-01 07:08:12,8.0,1900-01-01 07:00:08,"1,2","1,2",2.25
2,7,1N4AL2A,2,1900-01-01 07:14:37,12.0,1900-01-01 07:02:45,"1,2","1,2",2.25
3,8,KL4CJBS,2,1900-01-01 07:16:57,14.0,1900-01-01 07:02:47,"1,2","1,2",2.25
4,14,3D7TT2C,2,1900-01-01 07:15:18,7.0,1900-01-01 07:07:58,"1,2","1,2",2.25


In [201]:
# cleaning up df_merged

df_cleaned = df_merged[['id', 'plate_id', 'variable', 'value', 'time', 'toll_path', 'Toll']]

df_cleaned.head()

,id,plate_id,variable,value,time,toll_path,Toll
0,0,1N6AD0C,2,1900-01-01 07:11:02,11.0,"1,2",2.25
1,2,5LMJJ3J,2,1900-01-01 07:08:12,8.0,"1,2",2.25
2,7,1N4AL2A,2,1900-01-01 07:14:37,12.0,"1,2",2.25
3,8,KL4CJBS,2,1900-01-01 07:16:57,14.0,"1,2",2.25
4,14,3D7TT2C,2,1900-01-01 07:15:18,7.0,"1,2",2.25


In [203]:
# grouping by plate id and aggregating toll and time

df_agg = df_cleaned.groupby('plate_id').agg(toll_sum = ('Toll', 'sum'), time_sum = ('time', 'sum'))

df_agg.head()

,toll_sum,time_sum
plate_id,,
137FA90,6.25,131.0
137ZA90,6.25,157.0
19UUA56,6.25,88.0
19UUA65,7.50,95.0
19UUA66,13.75,187.0


In [204]:
df_agg.tail(10)

,toll_sum,time_sum
plate_id,,
YV1672M,15.25,218.0
YV1902F,12.50,249.0
YV1952A,7.50,129.0
YV1960B,18.00,196.0
YV1982B,6.25,150.0
YV426MD,7.50,100.0
YV4902D,12.50,170.0
YV4940B,6.25,121.0
YV4952C,13.75,264.0
